In [ ]:
# Import all libraries
# ML
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

# Utilities
from PIL import Image
import os
import numpy as np
from tqdm import tqdm

# Runtime
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
import os
from PIL import Image

# Create dataset, extended from torch
class SilverLineDataset(Dataset):
    def __init__(self, root_directory, transform=None):
        self.root_directory = root_directory
        self.transform = transform
        self.samples = []
        
        # For every image in silver_directory [CLASS 1]
        silver_directory = os.path.join(root_directory, "images/silver_line")
        
        for image_name in os.listdir(silver_directory):
            if image_name.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.samples.append((os.path.join(silver_directory, image_name), 1))
                    
        # For every image in silver_directory [CLASS 1]
        no_silver_directory = os.path.join(root_directory, "images/no_silver_line")
        
        for image_name in os.listdir(silver_directory):
            if image_name.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.samples.append((os.path.join(silver_directory, image_name), 0))
                    
    def __len__(self):
        # Return the number of items within
        return len(self.samples)
    
    def __getitem__(self, index: int):
        # Get an item from samples
        image_path, label = self.samples[index]
        image = Image.open(image_path).convert("RGB")
        
        if self.transform: image = self.transform(image)
        
        return image, label

In [ ]:
import toruch.nn as nn

class CustomSilverDetector(nn.Module):
    def __init__(self, input_size=64, class_count=2):
        super(CustomSilverDetector, self).__init__()
        
        """
        Define features
        - Using ReLU activation function
        
        """
        self.features = nn.Sequential(
            # Block 1: 64x64 -> 32x32
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            
            # Block 2: 32x32 -> 16x16
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            
            # Block 3: 16x16 -> 8x8
            nn.Conv2d(32, 48, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True),
            
            # Block 4: 8x8 -> 4x4
            nn.Conv2d(48, 64, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(32, class_count)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        
        return x

In [ ]:
import torch
from tqdm import tqdm

def train_model(model, train_loader, validation_loader, epoch_count, device="cuda"):
    # Setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    schedular = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
    
    train_losses =        []
    validation_losses =   []
    train_accuracies =      []
    validation_accuracies = []
    
    best_validation_accuracy = 0.0
    best_model_state = None
    
    # Train
    for epoch in range(epoch_count):
        model.train()
        
        train_loss    = 0.0
        train_correct = 0
        train_total   = 0
        
        train_bar = tqdm(train_loader, desc=f"[TRAINING] Epoch {epoch + 1}/{epoch_count}")
        
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backwards()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            train_bar.postfix({
                "Loss": f"{loss.item():.4f}",
                "Accuracy": f"{100 * train_correct / train_total:.2f}"
            })
            
        # Validation
        model.eval()
        
        validation_loss    = 0.0
        validation_correct = 0
        validation_total   = 0
        
        with torch.no_grad():
            for images, labels in validation_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
        # Calculate metrics
        train_accuracy =      100 *      train_correct / train_total
        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(train_accuracy)
        
        validation_accuracy = 100 * validation_correct / validation_total
        validation_losses.append(validation_loss / len(validation_loader))
        validation_accuracies.append(validation_accuracy)
        
        # Save the best model only
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_model_state = model.state_dict().copy()
            
            
        schedular.step()
        
        print(f"[RESULT] Epoch {epoch+1}: Train Acc: {train_accuracy:.2f}%, Val Acc: {validation_accuracy:.2f}%")
        
    # Load the best model
    model.load_state_dict(best_model_state)
    
    return model, train_losses, validation_losses, train_accuracies, validation_accuracies

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def evaluate_model(model, test_loader, device="cuda"):
    model.eval()
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = 100 * sum(np.array(all_predictions) == np.array(all_labels)) / len(all_labels)
    
    print(f"Test Accuracy: {accuracy:.2f}%")
    print("\nClassification Report:")
    print(classification_report(all_labels, all_predictions, target_names=["No Silver", "Silver"]))
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
                xticklabels=["No Silver", "Silver"],
                yticklabels=["No Silver", "Silver"])
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()
    
    return accuracy

In [ ]:
# Check which device is available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Data transforms for aggressive augmentation (small dataset)
train_transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Small input for speed
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset      = SilverLineDataset("/content/silver_cls/train", transform=train_transform)
validation_dataset = SilverLineDataset("/content/silver_cls/val",   transform=val_transform)
test_dataset       = SilverLineDataset("/content/silver_cls/test",  transform=val_transform)

print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(validation_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Create data loaders
train_loader =      DataLoader(train_dataset,      batch_size=16, shuffle=True,  num_workers=2)
validation_loader = DataLoader(validation_dataset, batch_size=16, shuffle=False, num_workers=2)
test_loader =       DataLoader(test_dataset,       batch_size=16, shuffle=False, num_workers=2)

# Initialize model
model = CustomSilverDetector(input_size=64, num_classes=2).to(device)

# Print model info
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

# Train model
model, train_losses, val_losses, train_accs, val_accs = train_model(
    model, train_loader, validation_loader, num_epochs=80, device=device
)



In [ ]:
# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label="Train Accuracy")
plt.plot(val_accs, label="Val Accuracy")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate on test set
test_accuracy = evaluate_model(model, test_loader, device)

# Save model for deployment
torch.save(model.state_dict(), "/content/custom_silver_detector.pth")